In [1]:
import requests
import json
from deep_translator import GoogleTranslator
import time
import traceback
import os
from os.path import exists
import copy
import hashlib
import unidecode
from datetime import datetime,date,timedelta
last_updated_string = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")


In [43]:
from bs4 import BeautifulSoup
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/100.0.1185.36"}
r =  requests.get("https://www.sfc.hk/en/alert-list" ,headers = headers)
soup = BeautifulSoup(r.content,"lxml")


In [52]:
soup.find("tbody" ,{"id":"alert-list-append-here"}).find_all("a",href=True)

<a href="https://www.sfc.hk/en/alert-list/2134">鴻運財富 (Chinese name only)</a>

In [8]:
def SourceDownload():
    res_indi = requests.get("https://sanctions-t.rnbo.gov.ua/apis/fizosoba/")
    res_ent = requests.get("https://sanctions-t.rnbo.gov.ua/apis/jurosoba/")
    data_indi = json.loads(res_indi.text)
    data_ent = json.loads(res_ent.text)
    data = data_indi+data_ent
    
    with open('inout-2.json', "w") as infile:
        infile.write(json.dumps(data,indent=2,ensure_ascii=False))
SourceDownload()

In [2]:
def langtranslation(to_translate):
    try:
        translated = GoogleTranslator(source='auto', target='en').translate(to_translate)
    except:
        try:
            translated = GoogleTranslator(source='auto', target='en').translate(to_translate)
        except:
            print(f">>>Translartion Bug : {to_translate}")
            translated = to_translate   
    return translated

# -*- coding: utf-8 -*-
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [16]:
# r = requests.get("https://sanctions-t.rnbo.gov.ua/apis/jurosoba/")
r = requests.get("https://sanctions-t.rnbo.gov.ua/apis/fizosoba/")
# with open("input.json","wb") as ip:
#     ip.write(r.content)


In [17]:
data = json.loads(r.text)
len(data)

3915

In [8]:
out_list = []
counter = 1
for i in data:
    uid = str(i["index"]) + "-" + str(i["ukaz_id"])
    names = []
    if i["name_ukr"]:
        names.append(i["name_ukr"])
    if i["name_original"]:
        names.append(i["name_original"])
    if i["name_alternative"]:
        names.append(i["name_alternative"])
    if len(names) > 0:
        name = names[0]
    else:
        name = ""
    print(f"{counter}---->{name}")
    counter+=1
    if name:
        if not isEnglish(name):
            name = langtranslation(name)
    else:
        continue #NOTE : there is no name no need to apped into list.
    
    place = i["place"]
    if place:
        if not isEnglish(place):
            place = langtranslation(place)
    else:
        place = ""
    
    adr = [{
        "complete_address" : place,
        "country" : ""
    }]

    if i["odrn_edrpou"]:
        reg_no = unidecode.unidecode(i["odrn_edrpou"])
    else:
        reg_no = ""
    if i["ipn"]:
        tax_no = unidecode.unidecode(i["ipn"])
    else:
        tax_no = ""
    
    docs = {
        "registartion_no" : reg_no,
        "Tax_Identification_Number" : tax_no
    }

    dec_no = i["ukaz_id"]
    if not dec_no:
        dec_no = ""
    dec_date = i["ukaz_date"]
    if not dec_date:
        dec_date = ""
    restr_end_date = i["restriction_end_date"]
    if not restr_end_date:
        restr_end_date = ""
    sanc_details = {
        "decree_no" : dec_no,
        "date_of_decree" : dec_date,
        "restriction_end_date" : restr_end_date
    }

    
    out_list.append({
        "uid": uid,
        "name": name,
        "alias_name" : names,
        "country" : [],
        "address" : adr,
        "last_updated": "last_updated_string",
        "nns_status": "False",
        "list_type" : "Entity",
        "entity_details" : {},
        "sanctions_details" : sanc_details,
        "documents":docs ,
        "list_id" :"UKR_T30090",
        "comment" : "",
        "sanction_list": {
            "sl_authority": "National Security and Defense Council, Ukraine",
            "sl_url": "https://sanctions-t.rnbo.gov.ua/",
            "sl_host_country": "Ukraine",
            "sl_type": "Sanctions",
            "watch_list": "European Watchlists",
            "sl_source": "National Security and Defence Council of Ukraine Sanctions List, Ukraine",
            "sl_description": "Sanctioned imposed by The National Security and Defense Council, Ukraine."
        } 
    })


1---->Товариство з обмеженою відповідальністю "НОВИНИ 24 ГОДИНИ"
2---->Товариство з обмеженою відповідальністю "Гринкран"
3---->Товариство з обмеженою відповідальністю "Сварка мостових конструкцій – Майстер"
4---->Товариство з обмеженою відповідальністю "Вантажна компанія "Новотранс"
5---->Товариство з обмеженою відповідальністю "Південна сталедротова компанія"
6---->Акціонерне товариство "Спеціальна виробничо-технічна база ЗВЬОЗДОЧКА"
7---->Товариство з обмеженою відповідальністю "НОВИЙ ФОРМАТ ТВ"
8---->Товариство з обмеженою відповідальністю "ЛІДЕР ТВ"
9---->Акціонерне товариство "РГ- Ремсервіс"
10---->Федеральна державна бюджетна освітня установа вищої освіти "Костромський державний університет"
11---->Акціонерне товариство "322 авіаційний ремонтний завод"
12---->ТОВ "ЛітРес"
13---->Товариство з обмеженою відповідальністю "ТВ ВИБІР"
14---->Товариство з обмеженою відповідальністю "ТЕЛЕРАДІОКОМПАНІЯ "112-ТВ"
15---->Товариство з обмеженою відповідальністю "Пелла-Маш"
16---->Акціонерне 

In [20]:
data[50]

{'index': 1536,
 'ukaz_id': '184-2020',
 'ukaz_date': '2020-05-14',
 'orderid': '150',
 'restriction_type': '1)    блокування активів - тимчасове обмеження права особи користуватися та розпоряджатися належним їй майном; 2)    заборона участі у приватизації, оренді державного майна резидентами іноземної держави та особами, які прямо чи опосередковано контролюються резидентами іноземної держави або діють в їх інтересах; 3)    відмова в наданні та скасування віз резидентам іноземних держав, застосування інших заборон в\'їзду на територію України; 4)    анулювання офіційних візитів, засідань, переговорів з питань укладення договорів чи угод; 5)    позбавлення державних нагород України, інших форм відзначення; 6)    інші санкції, що відповідають принципам їх застосування, встановленим Законом України "Про санкції" (заборона на встановлення ділових контактів; заборона діяльності на території України)',
 'restriction_period': 'Три роки',
 'restriction_end_date': '2023-05-14',
 'name_ukr': 'До

In [22]:
out_list_indi = []
counter = 1
for i in data:
    uid = str(i["index"]) + "-" + str(i["ukaz_id"])
    names = []
    if i["name_ukr"]:
        names.append(i["name_ukr"])
    if i["name_original"]:
        names.append(i["name_original"])
    if i["name_alternative"]:
        names.append(i["name_alternative"])
    if len(names) > 0:
        name = names[0]
    else:
        name = ""
    print(f"{counter}---->{name}")
    counter+=1
    if name:
        if not isEnglish(name):
            name = langtranslation(name)
    else:
        continue #NOTE : there is no name no need to apped into list.
    
    dob = []
    if i["birthdate"]:
        dob.append(i["birthdate"])
    
    citizenship = ""
    if i["citizenship"]:
        citizenship = i["citizenship"]

    place = i["livingplace"]
    if place:
        if not isEnglish(place):
            place = langtranslation(place)
    else:
        place = ""
    adr = [{
        "complete_address" : place,
        "country" : ""
    }]

    Bplace = i["birthplace"]
    if Bplace:
        if not isEnglish(Bplace):
            Bplace = langtranslation(Bplace)
    else:
        Bplace = ""
    Badr = [{
        "complete_address" : Bplace,
        "country" : ""
    }]
    
    idi_det = {
        "date_of_birth" : dob,
        "place_of_birth" : Badr
    } 

    # if i["odrn_edrpou"]:
    #     reg_no = unidecode.unidecode(i["odrn_edrpou"])
    # else:
    #     reg_no = ""
    # if i["ipn"]:
    #     tax_no = unidecode.unidecode(i["ipn"])
    # else:
    #     tax_no = ""
    
    # docs = {
    #     "registartion_no" : reg_no,
    #     "Tax_Identification_Number" : tax_no
    # }

    dec_no = i["ukaz_id"]
    if not dec_no:
        dec_no = ""
    dec_date = i["ukaz_date"]
    if not dec_date:
        dec_date = ""
    restr_end_date = i["restriction_end_date"]
    if not restr_end_date:
        restr_end_date = ""
    sanc_details = {
        "decree_no" : dec_no,
        "date_of_decree" : dec_date,
        "restriction_end_date" : restr_end_date
    }

    
    out_list_indi.append({
        "uid": uid,
        "name": name,
        "alias_name" : names,
        "country" : [],
        "address" : adr,
        "last_updated": last_updated_string,
        "nns_status": "False",
        "list_type" : "Individual",
        "individual_details" : idi_det,
        "sanctions_details" : sanc_details,
        "documents":{} ,
        "list_id" :"UKR_T30090",
        "comment" : "",
        "sanction_list": {
            "sl_authority": "National Security and Defense Council, Ukraine",
            "sl_url": "https://sanctions-t.rnbo.gov.ua/",
            "sl_host_country": "Ukraine",
            "sl_type": "Sanctions",
            "watch_list": "European Watchlists",
            "sl_source": "National Security and Defence Council of Ukraine Sanctions List, Ukraine",
            "sl_description": "Sanctioned imposed by The National Security and Defense Council, Ukraine."
        } 
    })


1---->Моляков Сергій Олександрович
2---->Царьов Олег Анатолійович
3---->Ліатсос Дімітріос 
4---->Скіппарі-Смірнова Дарія 
5---->Пантовіч Огнень
6---->Колісніченко Микола Петрович 
7---->Бєлік Юрій Леонідович
8---->Гірніченко Анастасія Олегівна
9---->Маріані Т’єррі
10---->Терещенко Юлія Володимирівна
11---->Кірюшин Валерій Іванович 
12---->Медведчук Віктор Володимирович
13---->Маслюк Олександр Миколайович
14---->Малихін Олександр Сергійович
15---->Шидловський Станіслав Валентинович
16---->Плотницький Ігор Венедиктович
17---->Дегтяренко Володимир Миколайович
18---->Івінська  Валентина  Леонтіївна
19---->Красних  Тетяна  Володимирівна
20---->Прокопчук Дмитро Володимирович
21---->Уварова Ірина Михайлівна
22---->Ценценатов Євген Михайлович
23---->Гордєєв Костянтин Миколайович
24---->Левченко Оксана Олександрівна
25---->Нікітін Василь Олександрович
26---->Шарагов Валерій Євгенович 
27---->Тимофєєв Олександр Юрійович
28---->Борщевський Євгеній Володимирович
29---->Таран  Станіслав  Степанович

In [1]:
out_list_indi[0]

NameError: name 'out_list_indi' is not defined

In [15]:
with open('ukr.json', "w",encoding='utf-8') as outfile:
    json.dump(out_list_indi, outfile,ensure_ascii=False,indent=2)


In [30]:
import pycountry
r = pycountry.countries
iso2Country = {}
for i in r:
    iso2Country[i.alpha_2] = i.name


In [29]:
iso2Country["RU"]

'Russian Federation'

In [54]:
import json
with open("diffrancefiles/ukraine-sanction-diffrance-13-4-2022.json" , "r") as f:
    data = json.load(f)


In [60]:
for i in data:
    del i["sanctions_details"]["restriction_end_date"]

In [61]:
data[0]

{'uid': '9a3246500bfdcba9f1f7c1384497b9e71a0dd2d4b2598dee2334f29f3864fa9b',
 'name': 'Limited Liability Company "NEWS 24 HOURS"',
 'alias_name': ['Товариство з обмеженою відповідальністю "НОВИНИ 24 ГОДИНИ"'],
 'country': [],
 'address': [{'complete_address': '', 'country': ''}],
 'last_updated': '2022-04-13T05:34:11',
 'nns_status': 'False',
 'list_type': 'Entity',
 'entity_details': {},
 'sanctions_details': {'decree_no': '43-2021', 'date_of_decree': '2021-02-02'},
 'documents': {'registartion_no': 'IeDRPOU33786517',
  'Tax_Identification_Number': ''},
 'list_id': 'UKR_T30090',
 'comment': '',
 'sanction_list': {'sl_authority': 'National Security and Defense Council, Ukraine',
  'sl_url': 'https://sanctions-t.rnbo.gov.ua/',
  'sl_host_country': 'Ukraine',
  'sl_type': 'Sanctions',
  'watch_list': 'European Watchlists',
  'sl_source': 'National Security and Defence Council of Ukraine Sanctions List, Ukraine',
  'sl_description': 'Sanctioned imposed by The National Security and Defense 

In [62]:
with open("ukraine-sanction-diffrance-13-4-2022.json" , "w") as f:
    json.dump(data,f,indent=2,ensure_ascii=False)
